<a href="https://colab.research.google.com/github/freddy-7/TI3002C/blob/main/9_Regresion_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [2]:
#Convertir archivo filtrado a CSV
df = pd.read_csv("valencia_trabajo.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9009 entries, 0 to 9008
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            9009 non-null   int64  
 1   listing_url                                   9009 non-null   object 
 2   scrape_id                                     9009 non-null   int64  
 3   last_scraped                                  9009 non-null   object 
 4   source                                        9009 non-null   object 
 5   name                                          9009 non-null   object 
 6   description                                   9009 non-null   object 
 7   neighborhood_overview                         9009 non-null   object 
 8   picture_url                                   9009 non-null   object 
 9   host_id                                       9009 non-null   i

In [3]:
escalar = StandardScaler()

In [12]:

df['host_response_rate'] = (
    df['host_response_rate']
    .astype(str)
    .str.replace('%', '', regex=False)
    .replace('nan', np.nan)
    .astype(float)
)

cols_bool = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 
             'has_availability', 'instant_bookable']

df[cols_bool] = df[cols_bool].replace({'t': 1, 'f': 0}).astype(int)


In [13]:
cols_bool = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
             'has_availability', 'instant_bookable']

# Reemplazar 't' y 'f' por 1 y 0 solo en esas columnas
df[cols_bool] = df[cols_bool].replace({'t': 1, 'f': 0})

print(df[cols_bool].head())

   host_is_superhost  host_has_profile_pic  host_identity_verified  \
0                  0                     1                       1   
1                  0                     1                       1   
2                  0                     1                       1   
3                  1                     1                       1   
4                  0                     1                       1   

   has_availability  instant_bookable  
0                 1                 0  
1                 1                 0  
2                 1                 0  
3                 1                 1  
4                 1                 0  


In [14]:
def entrenar_logit_univariada(y, X):
    """
    y: Serie binaria 0/1 (target)
    X: Serie numérica (predictor)
    """
    data = pd.concat([y, X], axis=1).dropna()
    y_clean = data.iloc[:,0].astype(int)
    X_clean = data.iloc[:,1].to_frame()

    # Split (estratificado como buena práctica)
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.2, random_state=None, stratify=y_clean
    )

    # Escalamiento SOLO a X
    escalar = StandardScaler()
    X_train = escalar.fit_transform(X_train)
    X_test  = escalar.transform(X_test)

    # Modelo
    modelo = LogisticRegression(max_iter=1000)
    modelo.fit(X_train, y_train)

    # Predicción
    y_pred = modelo.predict(X_test)

    # Métricas
    m_conf = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec  = recall_score(y_test, y_pred, zero_division=0)
    f1   = f1_score(y_test, y_pred, zero_division=0)

    print("Matriz de Confusión:")
    print(m_conf)
    print("\nPrecisión (accuracy):", acc)
    print("Precisión (precision):", prec)
    print("Sensibilidad (recall):", rec)
    print("Puntaje F1:", f1)

    return modelo, (acc, prec, rec, f1)
    

In [15]:
# Definir X y y antes de hacer el split
X = df[["host_response_rate"]]  
y = df["host_is_superhost"]    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
modelo_1, metrics_1 = entrenar_logit_univariada(y_train, X_train)

Matriz de Confusión:
[[905   0]
 [357   0]]

Precisión (accuracy): 0.7171156893819335
Precisión (precision): 0.0
Sensibilidad (recall): 0.0
Puntaje F1: 0.0


In [16]:
algoritmo_Pond2 = LogisticRegression(class_weight='balanced')
algoritmo_Pond2.fit(X_train, y_train)

y_pred_pond2 = algoritmo_Pond2.predict(X_test) 
y_pred_pond2

matriz = confusion_matrix(y_test, y_pred_pond2)
print('Matriz de Confusión:')
print(matriz)
print()

precision = precision_score(y_test, y_pred_pond2, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)
print()

exactitud = accuracy_score(y_test, y_pred_pond2)
print('Exactitud del modelo:')
print(exactitud)
print()

sensibilidad = recall_score(y_test, y_pred_pond2, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[ 564 1381]
 [  52  706]]

Precisión del modelo:
0.33828461907043605

Exactitud del modelo:
0.46984831668516464

Sensibilidad del modelo:
0.9313984168865436


In [17]:
y = df["host_has_profile_pic"]
X = df["review_scores_value"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
modelo_2, metrics_2 = entrenar_logit_univariada(y_train, X_train)

Matriz de Confusión:
[[   0   58]
 [   0 1204]]

Precisión (accuracy): 0.9540412044374009
Precisión (precision): 0.9540412044374009
Sensibilidad (recall): 1.0
Puntaje F1: 0.9764801297648013


In [18]:
algoritmo_Pond2 = LogisticRegression(class_weight='balanced')

# Asegura que X_train y X_test sean DataFrames
X_train = X_train.to_frame()
X_test = X_test.to_frame()

algoritmo_Pond2.fit(X_train, y_train)

y_pred_pond2 = algoritmo_Pond2.predict(X_test)
y_pred_pond2

matriz = confusion_matrix(y_test, y_pred_pond2)
print('Matriz de Confusión:')
print(matriz)
print()

precision = precision_score(y_test, y_pred_pond2, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)
print()

exactitud = accuracy_score(y_test, y_pred_pond2)
print('Exactitud del modelo:')
print(exactitud)
print()

sensibilidad = recall_score(y_test, y_pred_pond2, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[  26   78]
 [1038 1561]]

Precisión del modelo:
0.9524100061012812

Exactitud del modelo:
0.5871254162042175

Sensibilidad del modelo:
0.6006156213928434


In [19]:
y = df["has_availability"]
X = df["availability_365"]
modelo, metrics = entrenar_logit_univariada(y, X)

Matriz de Confusión:
[[   0   21]
 [   0 1781]]

Precisión (accuracy): 0.98834628190899
Precisión (precision): 0.98834628190899
Sensibilidad (recall): 1.0
Puntaje F1: 0.994138989673458


In [20]:
algoritmo_Pond2 = LogisticRegression(class_weight='balanced')
algoritmo_Pond2.fit(X_train, y_train)

y_pred_pond2 = algoritmo_Pond2.predict(X_test)
y_pred_pond2

matriz = confusion_matrix(y_test, y_pred_pond2)
print('Matriz de Confusión:')
print(matriz)
print()

precision = precision_score(y_test, y_pred_pond2, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)
print()

exactitud = accuracy_score(y_test, y_pred_pond2)
print('Exactitud del modelo:')
print(exactitud)
print()

sensibilidad = recall_score(y_test, y_pred_pond2, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad)

Matriz de Confusión:
[[  26   78]
 [1038 1561]]

Precisión del modelo:
0.9524100061012812

Exactitud del modelo:
0.5871254162042175

Sensibilidad del modelo:
0.6006156213928434


In [21]:
unico = np.unique(df['room_type'])
unico

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [22]:
df['room_type']= df['room_type'].replace(['Hotel room', 'Private room', 'Shared room'], "Not Entire home/apt")


In [23]:
Vars_Indep= df[['price', 'accommodates', 'bedrooms']]
Var_Dep= df['room_type']

X= Vars_Indep
y= Var_Dep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test) 
y_pred

from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)
print()

precision = precision_score(y_test, y_pred, average="binary", pos_label= "Entire home/apt")
print('Precisión del modelo:')
print(precision)
print()

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)
print()

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo:')
print(sensibilidad)
print()

puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="Entire home/apt")
print('Puntaje F1 del modelo:')
print(puntajef1)

Matriz de Confusión:
[[1806  127]
 [ 176  594]]

Precisión del modelo:
0.9112008072653885

Exactitud del modelo:
0.8879023307436182

Sensibilidad del modelo:
0.934299017071909

Puntaje F1 del modelo:
0.9226053639846743


In [15]:
unico = np.unique(df['host_response_rate'])
unico

array([  0.,   3.,   6.,   7.,   8.,   9.,  10.,  11.,  13.,  14.,  17.,
        18.,  20.,  24.,  25.,  27.,  30.,  31.,  33.,  36.,  39.,  40.,
        43.,  44.,  46.,  48.,  50.,  53.,  54.,  56.,  60.,  62.,  63.,
        64.,  67.,  68.,  69.,  70.,  71.,  73.,  75.,  77.,  78.,  79.,
        80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,
        91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.])

In [25]:
Vars_Indep = df[['host_is_superhost','review_scores_value','review_scores_rating']].astype(float)

umbral = df['host_response_rate'].median()
Var_Dep_cls = (df['host_response_rate'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[ 832   93]
 [1071  707]]

Exactitud: 0.5693673695893452

Precisión: 0.88375

Sensibilidad: 0.39763779527559057

F1: 0.5484871993793639


In [26]:
unico = np.unique(df['host_acceptance_rate'])
unico

array(['0', '0%', '10%', '100%', '11%', '13%', '14%', '15%', '16%', '17%',
       '18%', '19%', '2%', '20%', '21%', '22%', '23%', '24%', '25%',
       '26%', '27%', '29%', '30%', '31%', '33%', '34%', '35%', '36%',
       '37%', '38%', '39%', '4%', '40%', '41%', '42%', '43%', '44%',
       '45%', '46%', '47%', '48%', '49%', '5%', '50%', '51%', '52%',
       '53%', '54%', '55%', '56%', '57%', '58%', '59%', '6%', '60%',
       '61%', '62%', '63%', '64%', '65%', '66%', '67%', '68%', '69%',
       '7%', '70%', '71%', '72%', '73%', '74%', '75%', '76%', '77%',
       '78%', '79%', '8%', '80%', '81%', '82%', '83%', '84%', '85%',
       '86%', '87%', '88%', '89%', '9%', '90%', '91%', '92%', '93%',
       '94%', '95%', '96%', '97%', '98%', '99%'], dtype=object)

In [28]:
df['host_acceptance_rate'] = (
    df['host_acceptance_rate']
    .astype(str)                    # asegura que todos sean cadenas
    .str.replace('%', '', regex=False)  # elimina el símbolo %
    .replace('nan', np.nan)         # convierte el texto 'nan' a valor NaN
    .astype(float)                  # convierte a float
)


In [29]:
Vars_Indep = df[['host_is_superhost','review_scores_value','review_scores_rating']].astype(float)

umbral = df['host_acceptance_rate'].median()
Var_Dep_cls = (df['host_acceptance_rate'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[823 524]
 [626 730]]

Exactitud: 0.5745467998520163

Precisión: 0.5821371610845295

Sensibilidad: 0.5383480825958702

F1: 0.5593869731800766


In [30]:
Vars_Indep = df[['accommodates','bathrooms','bedrooms']].astype(float)

umbral = df['price'].median()
Var_Dep_cls = (df['price'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[ 848  348]
 [ 395 1112]]

Exactitud: 0.7251202367739549

Precisión: 0.7616438356164383

Sensibilidad: 0.7378898473788985

F1: 0.7495786990225818


In [31]:
Vars_Indep = df[['review_scores_cleanliness','review_scores_checkin','review_scores_value']].astype(float)

umbral = df['review_scores_rating'].median()
Var_Dep_cls = (df['review_scores_rating'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[ 839  286]
 [ 139 1439]]

Exactitud: 0.8427672955974843

Precisión: 0.8342028985507246

Sensibilidad: 0.9119138149556401

F1: 0.8713290947623372


In [32]:
Vars_Indep = df[['review_scores_cleanliness','review_scores_checkin','review_scores_rating']].astype(float)

umbral = df['review_scores_value'].median()
Var_Dep_cls = (df['review_scores_value'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[ 647  210]
 [ 238 1608]]

Exactitud: 0.8342582315945246

Precisión: 0.8844884488448845

Sensibilidad: 0.8710725893824486

F1: 0.8777292576419214


In [33]:
Vars_Indep = df[['bedrooms','bathrooms','price']].astype(float)

umbral = df['accommodates'].median()
Var_Dep_cls = (df['accommodates'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[1110  199]
 [ 337 1057]]

Exactitud: 0.8017018128005919

Precisión: 0.8415605095541401

Sensibilidad: 0.7582496413199427

F1: 0.7977358490566038


In [34]:
Vars_Indep = df[['accommodates','bathrooms','price']].astype(float)

umbral = df['bedrooms'].median()
Var_Dep_cls = (df['bedrooms'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[  94   50]
 [1251 1308]]

Exactitud: 0.5186829448760636

Precisión: 0.9631811487481591

Sensibilidad: 0.511137162954279

F1: 0.6678580546336482


In [39]:
umbral = df['bathrooms'].quantile(0.5)  # o 0.6, 0.4 según convenga
Var_Dep_cls = (df['bathrooms'] > umbral).astype(int)
print(df['bathrooms'].describe())
print(df['bathrooms'].isna().sum())
print(df['bathrooms'].unique())




count    9009.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: bathrooms, dtype: float64
0
[1.]


In [47]:
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')


In [49]:
print(df['bathrooms'].value_counts())
print("\nResumen estadístico:\n", df['bathrooms'].describe())


bathrooms
1.0    9009
Name: count, dtype: int64

Resumen estadístico:
 count    9009.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: bathrooms, dtype: float64


In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# --- Limpieza ---
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

Vars_Indep = df[['accommodates','bedrooms','price']].astype(float)

# Definir umbral (usa la mediana)
umbral = df['bathrooms'].median()
Var_Dep_cls = (df['bathrooms'] > umbral).astype(int)  # > en lugar de >=

print("\nConteo de clases:")
print(Var_Dep_cls.value_counts())

# Split
X_train, X_test, y_train, y_test = train_test_split(
    Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=42, stratify=Var_Dep_cls
)

# Escalamiento
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# Modelo
clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

# Predicción
y_pred = clf.predict(X_test)

# Métricas
matriz = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusión:\n", matriz)
print("Exactitud:", round(accuracy_score(y_test, y_pred), 3))
print("Precisión:", round(precision_score(y_test, y_pred), 3))
print("Sensibilidad:", round(recall_score(y_test, y_pred), 3))
print("F1:", round(f1_score(y_test, y_pred), 3))



Conteo de clases:
bathrooms
0    9009
Name: count, dtype: int64


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(0)

In [41]:
Vars_Indep = df[['bathrooms','bedrooms','price']].astype(float)

umbral = df['beds'].median()
Var_Dep_cls = (df['beds'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[ 851  157]
 [ 542 1153]]

Exactitud: 0.7413984461709212

Precisión: 0.8801526717557252

Sensibilidad: 0.68023598820059

F1: 0.76738768718802


In [42]:
Vars_Indep = df[['review_scores_value','review_scores_checkin','review_scores_rating']].astype(float)

umbral = df['review_scores_cleanliness'].median()
Var_Dep_cls = (df['review_scores_cleanliness'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[ 783  345]
 [ 178 1397]]

Exactitud: 0.8065112837587866

Precisión: 0.8019517795637199

Sensibilidad: 0.886984126984127

F1: 0.8423274042809767


In [43]:
Vars_Indep = df[['review_scores_value','review_scores_checkin','review_scores_rating']].astype(float)

umbral = df['review_scores_cleanliness'].median()
Var_Dep_cls = (df['review_scores_cleanliness'] >= umbral).astype(int)  # 1=Alto, 0=Bajo


X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep_cls, test_size=0.3, random_state=None, stratify=Var_Dep_cls)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)


clf = LogisticRegression(class_weight='balanced', max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz)
print()
print("Exactitud:", accuracy_score(y_test, y_pred))
print()
print("Precisión:", precision_score(y_test, y_pred, pos_label=1))
print()
print("Sensibilidad:", recall_score(y_test, y_pred, pos_label=1))
print()
print("F1:", f1_score(y_test, y_pred, pos_label=1))

Matriz de confusión:
 [[ 791  337]
 [ 201 1374]]

Exactitud: 0.8009618941916389

Precisión: 0.8030391583869082

Sensibilidad: 0.8723809523809524

F1: 0.8362751065124772
